In [4]:
# Module
import numpy as np

def sigmoid(x):
    return 1.0/(1.0+np.exp(-x))

print(sigmoid(-11)) 

1.670142184809518e-05


In [6]:
%run Single_Layer_Percept_전복고리수.ipynb

Epoch 1 : loss = 33.906, /t accuracy =0.557/0.801
Epoch 2 : loss = 8.225, /t accuracy =0.820/0.755
Epoch 3 : loss = 7.582, /t accuracy =0.812/0.865
Epoch 4 : loss = 7.476, /t accuracy =0.808/0.785
Epoch 5 : loss = 7.397, /t accuracy =0.810/0.820
Epoch 6 : loss = 7.331, /t accuracy =0.808/0.779
Epoch 7 : loss = 7.272, /t accuracy =0.808/0.810
Epoch 8 : loss = 7.220, /t accuracy =0.808/0.838
Epoch 9 : loss = 7.178, /t accuracy =0.810/0.838
Epoch 10 : loss = 7.138, /t accuracy =0.809/0.860

filnal Test : final accuracy = 0.860
[[1.03338961]
 [1.47724746]
 [1.67992711]
 [2.03717282]
 [1.60645146]
 [0.62703825]
 [2.42585997]
 [0.52521222]
 [0.46959183]
 [0.96029006]] [4.16916919]
Epoch 5 : loss = 6.807, /t accuracy =0.809/0.808
Epoch 10 : loss = 6.474, /t accuracy =0.813/0.794
Epoch 15 : loss = 6.237, /t accuracy =0.818/0.818
Epoch 20 : loss = 6.018, /t accuracy =0.821/0.811
Epoch 25 : loss = 5.843, /t accuracy =0.824/0.805
Epoch 30 : loss = 5.696, /t accuracy =0.826/0.811
Epoch 35 : loss =

In [40]:
# 메인함수 정의
def pulsar_exec(epoch_count=10,mb_size=10,report=1):
    load_pulsar_dataset()
    init_model()
    train_test(epoch_count,mb_size,report)

In [68]:
# 데이터 적재함수 정의
def load_pulsar_dataset():
    with open("pulsar_data_train.csv") as csvfile:
        csvreader=csv.reader(csvfile)
        next(csvreader,None)
        rows=[]

        for row in csvreader:
            for i in range(0,len(row)) :
                if row[i]=='':row[i]=0
            rows.append(row)

    global data, input_cnt, output_cnt
    input_cnt, output_cnt = 8,1
    data = np.asarray(rows,dtype=np.float32)


In [69]:
# a=['15','12','11','10','3','2','']
# for i in range(0,len(a)): 
#     if a[i]== '': a[i] = 0
# #     print(a[i])

# a1=np.asarray(a,dtype=np.float32)

# a1.dtype
data[0]

array(['121.15625', '48.37297113', '0.375484665', '-0.013165488999999999',
       '3.168896321', '18.399366600000004', '7.449874148999999',
       '65.15929771', '0.0'], dtype='<U23')

In [70]:
#  후처리 과정에 대한 순전파와 역전파 함수 재정의
def forward_postproc(output,y):
    entropy = sigmoid_cross_entropy_with_logits(y,output)
    loss=np.mean(entropy)
    return loss,[y,output,entropy]

def backprop_postproc(G_loss,aux):
    y,output,entropy=aux
    
    g_loss_entropy=1.0/np.prod(entropy.shape)
    g_entropy_output=sigmoid_cross_entropy_with_logits_derv(y,output)
    
    G_entropy = g_loss_entropy * G_loss
    G_output = g_entropy_output * G_entropy
    
    return G_output

In [74]:
# 정확도 함수 재정의
def eval_accuracy(output,y):
    estimate = np.greater(output,0)
    answer=np.greater(y,0.5)
    correct = np.equal(estimate,answer)
    
    return np.mean(correct)

In [83]:
# 시그모이드 관련 함수
def relu(x):
    return np.maximum(x,0)

def sigmoid(x):
    return np.exp(-relu(-x))/ (1 + np.exp(-np.abs(x)))

def sigmoid_derv(x,y):
    return y*(1-y)

def sigmoid_cross_entropy_with_logits(z,x):
    return relu(x)-x*z+np.log(1+np.exp(-np.abs(x)))

def sigmoid_cross_entorpy_with_logits_derv(z,x):
    
    return -z + sigmoid(x)

In [84]:
pulsar_exec()

Epoch 1 : loss = 11.012, /t accuracy =0.945/1.000
Epoch 2 : loss = 7.027, /t accuracy =0.951/1.000
Epoch 3 : loss = 8.520, /t accuracy =0.952/1.000
Epoch 4 : loss = 8.772, /t accuracy =0.956/1.000
Epoch 5 : loss = 9.666, /t accuracy =0.954/0.900
Epoch 6 : loss = 9.345, /t accuracy =0.956/1.000
Epoch 7 : loss = 7.096, /t accuracy =0.959/1.000
Epoch 8 : loss = 9.849, /t accuracy =0.956/1.000
Epoch 9 : loss = 8.784, /t accuracy =0.956/1.000
Epoch 10 : loss = 8.787, /t accuracy =0.955/1.000

filnal Test : final accuracy = 1.000
